# LLM Optimization Modelling Experiment

In [156]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [508]:
problem = '''You are tasked with scheduling the power output of 6 electric power thermal units over the timespan of 15 periods. There is a constant startup cost of for each power unit that is applied if the power plant is turned on. The startup costs are 10324€, 5678€, 7802€, 12899€, 4596€ and 9076€ for powerplants 1 to 6, respectively. In addition, there is a constant shutdown cost for each power unit that is applied if the power plant is turned off. The shutdown costs are 2673€, 5893€, 982€, 6783€, 2596€ and 3561€ for powerplants 1 to 6, respectively. There is also a fixed and variable cost applied if the power plant is running. The fixed cost is constant and the variable cost is proportional to the output of a power plant. There are lower and upper bounds for the output power for each unit. The variable costs are different for each power plant and at each time step. Next, there are maximum power increments and decrements for each power plant that limit how much the output can change from one time period to the next. There is a total power demand that needs to be fulfilled by the power units. Finally, for security reasons, the total available power out should always be 10% higher than the demand. 
'''

## 2. Generate the mathematical model

In [509]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [510]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Sets and Indices:**

*  $i$ = Index for power plants, $i = 1, 2, ..., 6$
*  $t$ = Index for time periods, $t = 1, 2, ..., 15$

**Parameters:**

* $SU_i$ = Startup cost for power plant $i$ (€)
    * $SU_1 = 10324$, $SU_2 = 5678$, $SU_3 = 7802$, $SU_4 = 12899$, $SU_5 = 4596$, $SU_6 = 9076$
* $SD_i$ = Shutdown cost for power plant $i$ (€)
    * $SD_1 = 2673$, $SD_2 = 5893$, $SD_3 = 982$, $SD_4 = 6783$, $SD_5 = 2596$, $SD_6 = 3561$
* $FC_i$ = Fixed cost for running power plant $i$ (€) 
* $VC_{i,t}$ = Variable cost per unit output for power plant $i$ at time $t$ (€/MWh)
* $P_{i}^{min}$ = Minimum power output for power plant $i$ (MW)
* $P_{i}^{max}$ = Maximum power output for power plant $i$ (MW)
* $\Delta P_{i}^{up}$ = Maximum power increment for power plant $i$ (MW)
* $\Delta P_{i}^{down}$ = Maximum power decrement for power plant $i$ (MW)
* $D_t$ = Total power demand at time $t$ (MW)

**Decision Variables:**

* $p_{i,t}$ = Power output of power plant $i$ at time $t$ (MW)
* $u_{i,t}$ = Binary variable, 1 if power plant $i$ is running at time $t$, 0 otherwise
* $s_{i,t}$ = Binary variable, 1 if power plant $i$ is started up at time $t$, 0 otherwise
* $d_{i,t}$ = Binary variable, 1 if power plant $i$ is shut down at time $t$, 0 otherwise

**Objective Function:**

Minimize the total cost over the planning horizon:

$$
\min \sum_{i=1}^{6} \sum_{t=1}^{15} (FC_i \cdot u_{i,t} + VC_{i,t} \cdot p_{i,t} + SU_i \cdot s_{i,t} + SD_i \cdot d_{i,t})
$$

**Constraints:**

1. **Power balance:** The total power output must meet the demand at each time period, plus a 10% safety margin.
   $$
   \sum_{i=1}^{6} p_{i,t} \ge 1.1 \cdot D_t, \forall t
   $$

2. **Power output limits:** The output of each power plant must be within its limits.
   $$
   P_{i}^{min} \cdot u_{i,t} \le p_{i,t} \le P_{i}^{max} \cdot u_{i,t}, \forall i, \forall t
   $$

3. **Ramp rate limits:** The change in power output between consecutive time periods must be within the ramp rate limits.
   $$
   -\Delta P_{i}^{down} \le p_{i,t} - p_{i,t-1} \le \Delta P_{i}^{up}, \forall i, \forall t
   $$

4. **Startup and shutdown logic:**  The startup and shutdown variables are linked to the unit commitment status.
   $$
   s_{i,t} \ge u_{i,t} - u_{i,t-1}, \forall i, \forall t
   $$
   $$
   d_{i,t} \ge u_{i,t-1} - u_{i,t}, \forall i, \forall t 
   $$

5. **Initial conditions:** Define the initial status of each power plant (on or off).  For example:
   $$
   u_{i,0} = 0, \forall i
   $$

This model can be implemented and solved using optimization software like CPLEX, Gurobi, or other suitable solvers. 


## 3. Generate the pyomo code

In [511]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [512]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 1000, 2: 800, 3: 1200, 4: 1500, 5: 900, 6: 1100} # Sample data
VC = {(i, t): 0.1 * i * t for i in range(1, 7) for t in range(1, 16)} # Sample data
P_min = {1: 10, 2: 8, 3: 15, 4: 20, 5: 12, 6: 18} # Sample data
P_max = {1: 50, 2: 40, 3: 60, 4: 80, 5: 45, 6: 70} # Sample data
Delta_P_up = {1: 20, 2: 15, 3: 25, 4: 30, 5: 22, 6: 28} # Sample data
Delta_P_down = {1: 18, 2: 13, 3: 22, 4: 28, 5: 20, 6: 25} # Sample data
D = {t: 100 + 10 * t for t in range(1, 16)} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Power plants
model.T = pyo.Set(initialize=range(1, 16)) # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.Delta_P_up = pyo.Param(model.I, initialize=Delta_P_up)
model.Delta_P_down = pyo.Param(model.I, initialize=Delta_P_down)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.p = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals) # Power output
model.u = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment status
model.s = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.d = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.u[i, t] + model.VC[i, t] * model.p[i, t] + model.SU[i] * model.s[i, t] + model.SD[i] * model.d[i, t] for i in model.I for t in model.T)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.p[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_limits_rule(model, i, t):
    return model.P_min[i] * model.u[i, t] <= model.p[i, t] <= model.P_max[i] * model.u[i, t]
model.power_output_limits = pyo.Constraint(model.I, model.T, rule=power_output_limits_rule)

# Ramp rate limits
def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] <= model.Delta_P_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] >= -model.Delta_P_down[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

# Startup and shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.s[i, t] >= model.u[i, t] - model.u[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.d[i, t] >= model.u[i, t-1] - model.u[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Initial conditions (all plants off at time 0)
def initial_status_rule(model, i):
    return model.u[i, 1] == 0 
model.initial_status = pyo.Constraint(model.I, rule=initial_status_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # You can choose a different solver like 'gurobi' or 'cplex'
solver.solve(model)

# Print the results
print("Optimal objective value:", model.objective())
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Power plant {i}: Output = {model.p[i, t]()}, Status = {'On' if model.u[i, t]() == 1 else 'Off'}")

```


## 4. Input problem data and try running the generated code

In [516]:
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 1000, 2: 800, 3: 1200, 4: 1500, 5: 900, 6: 1100} # Sample data
VC = {(i, t): 0.1 * i * t for i in range(1, 7) for t in range(1, 16)} # Sample data
P_min = {1: 10, 2: 8, 3: 15, 4: 20, 5: 12, 6: 18} # Sample data
P_max = {1: 50, 2: 40, 3: 60, 4: 80, 5: 45, 6: 70} # Sample data
Delta_P_up = {1: 20, 2: 15, 3: 25, 4: 30, 5: 22, 6: 28} # Sample data
Delta_P_down = {1: 18, 2: 13, 3: 22, 4: 28, 5: 20, 6: 25} # Sample data
D = {t: 100 + 10 * t for t in range(1, 16)} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Power plants
model.T = pyo.Set(initialize=range(1, 16)) # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.Delta_P_up = pyo.Param(model.I, initialize=Delta_P_up)
model.Delta_P_down = pyo.Param(model.I, initialize=Delta_P_down)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.p = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals) # Power output
model.u = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment status
model.s = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.d = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.u[i, t] + model.VC[i, t] * model.p[i, t] + model.SU[i] * model.s[i, t] + model.SD[i] * model.d[i, t] for i in model.I for t in model.T)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.p[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_limits_rule(model, i, t):
    return model.P_min[i] * model.u[i, t] <= model.p[i, t] <= model.P_max[i] * model.u[i, t]
model.power_output_limits = pyo.Constraint(model.I, model.T, rule=power_output_limits_rule)

# Ramp rate limits
def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] <= model.Delta_P_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] >= -model.Delta_P_down[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

# Startup and shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.s[i, t] >= model.u[i, t] - model.u[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.d[i, t] >= model.u[i, t-1] - model.u[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# Initial conditions (all plants off at time 0)
def initial_status_rule(model, i):
    return model.u[i, 1] == 0 
model.initial_status = pyo.Constraint(model.I, rule=initial_status_rule)

# Solve the model
solver = pyo.SolverFactory('cbc') # You can choose a different solver like 'gurobi' or 'cplex'
solver.solve(model)

# Print the results
print("Optimal objective value:", model.objective())
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Power plant {i}: Output = {model.p[i, t]()}, Status = {'On' if model.u[i, t]() == 1 else 'Off'}")

ERROR: Rule failed when generating expression for Constraint
power_output_limits with index (1, 1): PyomoException: Cannot convert non-
constant Pyomo expression (10*u[1,1]  <=  p[1,1]) to bool. This error is
usually caused by using a Var, unit, or mutable Param in a Boolean context
such as an "if" statement, or when checking container membership or equality.
For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
        ...     pass
    and
        >>> m.y = Var()
        >>> if m.y in [m.x, m.y]:
        ...     pass
    would both cause this exception.
ERROR: Constructing component 'power_output_limits' from data=None failed:
        PyomoException: Cannot convert non-constant Pyomo expression
        (10*u[1,1]  <=  p[1,1]) to bool.
    This error is usually caused by using a Var, unit, or mutable Param in a
    Boolean context such as an "if" statement, or when checking container
    membership or equality. For example,
        >>> m.x = Var()
        >>> if m.x >= 1:
     

PyomoException: Cannot convert non-constant Pyomo expression (10*u[1,1]  <=  p[1,1]) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

## 5. Correct the code to verify model viability (optional)

In [539]:
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 2000, 2: 3000, 3: 2500, 4: 4000, 5: 3500, 6: 4500} # Sample data
VC = {(1, 1): 20, (1, 2): 22, (1, 3): 23, (1, 4): 24, (1, 5): 25, (1, 6): 26, (1, 7): 27, (1, 8): 28, (1, 9): 29, (1, 10): 30, (1, 11): 31, (1, 12): 32, (1, 13): 33, (1, 14): 34, (1, 15): 35, (2, 1): 15, (2, 2): 16, (2, 3): 17, (2, 4): 18, (2, 5): 19, (2, 6): 20, (2, 7): 21, (2, 8): 22, (2, 9): 23, (2, 10): 24, (2, 11): 25, (2, 12): 26, (2, 13): 27, (2, 14): 28, (2, 15): 29, (3, 1): 18, (3, 2): 19, (3, 3): 20, (3, 4): 21, (3, 5): 22, (3, 6): 23, (3, 7): 24, (3, 8): 25, (3, 9): 26, (3, 10): 27, (3, 11): 28, (3, 12): 29, (3, 13): 30, (3, 14): 31, (3, 15): 32, (4, 1): 25, (4, 2): 26, (4, 3): 27, (4, 4): 28, (4, 5): 29, (4, 6): 30, (4, 7): 31, (4, 8): 32, (4, 9): 33, (4, 10): 34, (4, 11): 35, (4, 12): 36, (4, 13): 37, (4, 14): 38, (4, 15): 39, (5, 1): 22, (5, 2): 23, (5, 3): 24, (5, 4): 25, (5, 5): 26, (5, 6): 27, (5, 7): 28, (5, 8): 29, (5, 9): 30, (5, 10): 31, (5, 11): 32, (5, 12): 33, (5, 13): 34, (5, 14): 35, (5, 15): 36, (6, 1): 30, (6, 2): 31, (6, 3): 32, (6, 4): 33, (6, 5): 34, (6, 6): 35, (6, 7): 36, (6, 8): 37, (6, 9): 38, (6, 10): 39, (6, 11): 40, (6, 12): 41, (6, 13): 42, (6, 14): 43, (6, 15): 44} # Sample data
P_min = {1: 50, 2: 40, 3: 30, 4: 60, 5: 55, 6: 65} # Sample data
P_max = {1: 500, 2:  600, 3: 550, 4: 700, 5: 650, 6: 750} # Sample data
Delta_P_up = {1: 100, 2: 120, 3: 110, 4: 130, 5: 125, 6: 140} # Sample data
Delta_P_down = {1: 90, 2: 110, 3: 100, 4: 120, 5: 115, 6: 130} # Sample data
D = {1: 1000, 2: 1200, 3: 1300, 4: 1100, 5: 1500, 6: 1400, 7: 1600, 8: 1300, 9: 1700, 10: 1800,
     11: 1900, 12: 1600, 13: 2000, 14: 1800, 15: 1700}# Sample data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Power plants
model.T = pyo.Set(initialize=range(1, 16)) # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.I, initialize=FC)
model.VC = pyo.Param(model.I, model.T, initialize=VC)
model.P_min = pyo.Param(model.I, initialize=P_min)
model.P_max = pyo.Param(model.I, initialize=P_max)
model.Delta_P_up = pyo.Param(model.I, initialize=Delta_P_up)
model.Delta_P_down = pyo.Param(model.I, initialize=Delta_P_down)
model.D = pyo.Param(model.T, initialize=D)

# Decision variables
model.p = pyo.Var(model.I, model.T, within=pyo.NonNegativeReals) # Power output
model.u = pyo.Var(model.I, model.T, within=pyo.Binary)  # Unit commitment status
model.s = pyo.Var(model.I, model.T, within=pyo.Binary)  # Startup status
model.d = pyo.Var(model.I, model.T, within=pyo.Binary)  # Shutdown status

# Objective function
def objective_rule(model):
    return sum(model.FC[i] * model.u[i, t] + model.VC[i, t] * model.p[i, t] + model.SU[i] * model.s[i, t] + model.SD[i] * model.d[i, t] for i in model.I for t in model.T)
model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
# Power balance
def power_balance_rule(model, t):
    return sum(model.p[i, t] for i in model.I) >= 1.1 * model.D[t]
model.power_balance = pyo.Constraint(model.T, rule=power_balance_rule)

# Power output limits
def power_output_min_rule(model, i, t):
    return model.p[i, t] >= model.P_min[i] * model.u[i, t]
model.power_output_min = pyo.Constraint(model.I, model.T, rule=power_output_min_rule)

def power_output_max_rule(model, i, t):
    return model.p[i, t] <= model.P_max[i] * model.u[i, t]
model.power_output_max = pyo.Constraint(model.I, model.T, rule=power_output_max_rule)

# Ramp rate limits
def ramp_rate_up_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] <= model.Delta_P_up[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_up = pyo.Constraint(model.I, model.T, rule=ramp_rate_up_rule)

def ramp_rate_down_rule(model, i, t):
    if t > 1:
        return model.p[i, t] - model.p[i, t-1] >= -model.Delta_P_down[i]
    else:
        return pyo.Constraint.Skip
model.ramp_rate_down = pyo.Constraint(model.I, model.T, rule=ramp_rate_down_rule)

# Startup and shutdown logic
def startup_rule(model, i, t):
    if t > 1:
        return model.s[i, t] >= model.u[i, t] - model.u[i, t-1]
    else:
        return pyo.Constraint.Skip
model.startup = pyo.Constraint(model.I, model.T, rule=startup_rule)

def shutdown_rule(model, i, t):
    if t > 1:
        return model.d[i, t] >= model.u[i, t-1] - model.u[i, t]
    else:
        return pyo.Constraint.Skip
model.shutdown = pyo.Constraint(model.I, model.T, rule=shutdown_rule)

# # Initial conditions (all plants off at time 0)
# def initial_status_rule(model, i):
#     return model.u[i, 1] == 0 
# model.initial_status = pyo.Constraint(model.I, rule=initial_status_rule)

# Solve the model
solver = pyo.SolverFactory('glpk') # You can choose a different solver like 'gurobi' or 'cplex'
solver.solve(model)

# Print the results
#print("Optimal objective value:", model.objective())
for t in model.T:
    print(f"Time period {t}:")
    for i in model.I:
        print(f"  Power plant {i}: Output = {model.p[i, t]()}, Status = {'On' if model.u[i, t]() == 1 else 'Off'}")

Time period 1:
  Power plant 1: Output = 100.0, Status = On
  Power plant 2: Output = 590.0, Status = On
  Power plant 3: Output = 410.0, Status = On
  Power plant 4: Output = 0.0, Status = Off
  Power plant 5: Output = 0.0, Status = Off
  Power plant 6: Output = 0.0, Status = Off
Time period 2:
  Power plant 1: Output = 200.0, Status = On
  Power plant 2: Output = 600.0, Status = On
  Power plant 3: Output = 520.0, Status = On
  Power plant 4: Output = 0.0, Status = Off
  Power plant 5: Output = 0.0, Status = Off
  Power plant 6: Output = 0.0, Status = Off
Time period 3:
  Power plant 1: Output = 300.0, Status = On
  Power plant 2: Output = 590.0, Status = On
  Power plant 3: Output = 540.0, Status = On
  Power plant 4: Output = 0.0, Status = Off
  Power plant 5: Output = 0.0, Status = Off
  Power plant 6: Output = 0.0, Status = Off
Time period 4:
  Power plant 1: Output = 400.0, Status = On
  Power plant 2: Output = 480.0, Status = On
  Power plant 3: Output = 440.0, Status = On
  Po

In [541]:
model.objective()

802166.0

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [542]:
print(response.text)

## Mathematical Optimization Model for Power Plant Scheduling

**Sets and Indices:**

*  $i$ = Index for power plants, $i = 1, 2, ..., 6$
*  $t$ = Index for time periods, $t = 1, 2, ..., 15$

**Parameters:**

* $SU_i$ = Startup cost for power plant $i$ (€)
    * $SU_1 = 10324$, $SU_2 = 5678$, $SU_3 = 7802$, $SU_4 = 12899$, $SU_5 = 4596$, $SU_6 = 9076$
* $SD_i$ = Shutdown cost for power plant $i$ (€)
    * $SD_1 = 2673$, $SD_2 = 5893$, $SD_3 = 982$, $SD_4 = 6783$, $SD_5 = 2596$, $SD_6 = 3561$
* $FC_i$ = Fixed cost for running power plant $i$ (€) 
* $VC_{i,t}$ = Variable cost per unit output for power plant $i$ at time $t$ (€/MWh)
* $P_{i}^{min}$ = Minimum power output for power plant $i$ (MW)
* $P_{i}^{max}$ = Maximum power output for power plant $i$ (MW)
* $\Delta P_{i}^{up}$ = Maximum power increment for power plant $i$ (MW)
* $\Delta P_{i}^{down}$ = Maximum power decrement for power plant $i$ (MW)
* $D_t$ = Total power demand at time $t$ (MW)

**Decision Variables:**

* $p_{i,t}$ = P

In [543]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample data (replace with actual data if available)
SU = {1: 10324, 2: 5678, 3: 7802, 4: 12899, 5: 4596, 6: 9076}
SD = {1: 2673, 2: 5893, 3: 982, 4: 6783, 5: 2596, 6: 3561}
FC = {1: 1000, 2: 800, 3: 1200, 4: 1500, 5: 900, 6: 1100} # Sample data
VC = {(i, t): 0.1 * i * t for i in range(1, 7) for t in range(1, 16)} # Sample data
P_min = {1: 10, 2: 8, 3: 15, 4: 20, 5: 12, 6: 18} # Sample data
P_max = {1: 50, 2: 40, 3: 60, 4: 80, 5: 45, 6: 70} # Sample data
Delta_P_up = {1: 20, 2: 15, 3: 25, 4: 30, 5: 22, 6: 28} # Sample data
Delta_P_down = {1: 18, 2: 13, 3: 22, 4: 28, 5: 20, 6: 25} # Sample data
D = {t: 100 + 10 * t for t in range(1, 16)} # Sample data

# Model creation
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=range(1, 7))  # Power plants
model.T = pyo.Set(initialize=range(1, 16)) # Time periods

# Parameters
model.SU = pyo.Param(model.I, initialize=SU)
model.SD = pyo.Param(model.I, initialize=SD)
model.FC = pyo.Param(model.